In [2]:
%load_ext autoreload
%autoreload 2

import seaborn as sns
sns.set_context('talk')
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%config InlineBackend.figure_format = 'retina'
from datetime import timedelta
import datetime as dt
import scipy
import matplotlib.dates as mdates
import matplotlib.cbook as cbook
from matplotlib.dates import MonthLocator, WeekdayLocator, DateFormatter
import pickle

from get_data.get_euromomo_data import get_euromomo_data
from get_data.get_data import get_ecdc_data,\
                        get_data_UK,\
                        get_data_Spain, \
                        get_data_Austria, \
                        get_data_Malta, \
                        get_data_Italy, \
                        get_data_Belgium,\
                        get_data_France, \
                        get_data_Portugal, \
                        get_data_Berlin,\
                        get_data_Norway,\
                        get_data_Netherlands,\
                        get_data_Denmark,\
                        get_data_Switzerland,\
                        get_data_Ireland,\
                        get_data_Greece,\
                        get_data_Luxemburg,\
                        get_data_Estonia,\
                        get_data_Sweden,\
                        get_data_Hesse,\
                        get_data_Slovenia

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Some of examples of web scraping using modules in this package

## Get ECDC data and group all Europe

In [3]:
ecdc_data=get_ecdc_data(continent='Europe')
ecdc_europe=ecdc_data.groupby(by='date',as_index=False).sum()
ecdc_europe.deaths.clip(0,inplace=True)
ecdc_europe['state']='Europe'
ecdc_europe['datasource']='ecdc.europa.eu'




## Print list of countries in ECDC Europe

In [4]:

from pprint import pprint
countries=(str(ecdc_data.state.unique())).strip('[]')
print('Countries in this data set:\n', countries)


Countries in this data set:
 'Albania' 'Andorra' 'Armenia' 'Austria' 'Azerbaijan' 'Belarus' 'Belgium'
 'Bosnia and Herzegovina' 'Bulgaria' 'Croatia' 'Cyprus' 'Czech Republic'
 'Denmark' 'Estonia' 'Faroe_Islands' 'Finland' 'France' 'Georgia'
 'Germany' 'Gibraltar' 'Greece' 'Guernsey' 'Holy_See' 'Hungary' 'Iceland'
 'Ireland' 'Isle_of_Man' 'Italy' 'Jersey' 'Kosovo' 'Latvia'
 'Liechtenstein' 'Lithuania' 'Luxembourg' 'Malta' 'Moldova' 'Monaco'
 'Montenegro' 'Netherlands' 'North_Macedonia' 'Norway' 'Poland' 'Portugal'
 'Romania' 'Russia' 'San_Marino' 'Serbia' 'Slovakia' 'Slovenia' 'Spain'
 'Sweden' 'Switzerland' 'Turkey' 'Ukraine' 'United Kingdom'


In [10]:
import json
import requests
pd.read_csv()

<Response [404]>

# Get Covid data for individual Euromomo countries

In [12]:
countries=['Austria', 'Belgium','Cyprus', 'Denmark', 'Estonia', 'Finland', 'France','Germany (Berlin)',
           'Germany (Hesse)','Greece', 'Hungary','Ireland', 'Italy', 'Luxembourg', 'Malta',
           'Netherlands', 'Norway', 'Portugal','Slovenia','Spain', 'Sweden', 
           'Switzerland', 'UK']
 



FI=ecdc_data[ecdc_data.state=='Finland']
HU=ecdc_data[ecdc_data.state=='Hungary']
CY=ecdc_data[ecdc_data.state=='Cyprus']

ES=get_data_Spain()
AT=get_data_Austria()
MT=get_data_Malta()
IT=get_data_Italy()
BE=get_data_Belgium()
FR=get_data_France()
PT=get_data_Portugal()
Berlin=get_data_Berlin()
NO=get_data_Norway()
NL=get_data_Netherlands()
DK=get_data_Denmark()
CH=get_data_Switzerland()
IE=get_data_Ireland()
GR=get_data_Greece()
LU=get_data_Luxemburg()
EE=get_data_Estonia()
SE=get_data_Sweden()
Hesse=get_data_Hesse()
UK=get_data_UK()
SO=get_data_Slovenia()

### Make a single dataframe containing Covid data for all EuroMOMO countries

In [ ]:


df=pd.concat([FI,HU,CY,ES,AT,MT,IT,BE,FR,PT,Berlin,
              NO,NL,DK,CH,IE,GR,LU,EE,SE,SO,Hesse,UK])


if len(countries)==len(df.state.unique()):
    print('All Euromomo countries done')
else: print('Something is wrong')

# Convert all timestamps to same time zone
df['date']=pd.to_datetime(df.date,utc=True)


In [ ]:
# Inspect dataframe 
df

In [ ]:
# Get list of countries and datasource
datasources=df.drop_duplicates('state',ignore_index=True)
datasources=datasources[['state','datasource']]
datasources=datasources.sort_values(by='state').reset_index()
# Get only relevant columns

## Output the list of data sources

fig, ax = plt.subplots(figsize=(12, 5)) # set size frame
ax.xaxis.set_visible(False)  # hide the x axis
ax.yaxis.set_visible(False)  # hide the y axis
ax.set_frame_on(False) 

fig=pd.plotting.table(ax,datasources[['state','datasource']],loc='upper right')  # where df is your data frame
fig.set_fontsize(30)

plt.savefig('plots/Euromomo_data_sources.png',dpi=200)



In [ ]:

# Now group by date
df=df[['date','positive','deaths','total']].groupby(by='date',as_index=False).sum()

In [ ]:
df.tail()


# Get EU/EEA & UK test and other data

In [ ]:
# Get EU/EEA & UK Tests

from get_data.get_data_ecdc import get_data_ecdc_tests

EUtestdf=get_data_ecdc_tests()

# Get other data only for only EU/EEA & UK 
ecdc_EUtest=ecdc_data[ecdc_data.state.isin(list(EUtestdf.country.unique()))]

#from pprint import pprint
countries=(str(ecdc_EUtest.state.unique())).strip('[]')
print('EU cases & deaths: Countries in this data set:\n', countries)

ecdc_EUtest=ecdc_EUtest.groupby(by='date',as_index=False).sum()
ecdc_EUtest.deaths.clip(0,inplace=True)
ecdc_EUtest['state']='EU/EEA & UK'
ecdc_EUtest['datasource']='ecdc.europa.eu'
ecdc_EUtest['positive_avg']=ecdc_EUtest.positive.rolling(window=7,center=True).mean()

print('EU tests: Countries in this data set:\n', (str(EUtestdf.country.unique())).strip('[]'))

#EUtest['date']=pd.to_datetime(EUtest.date)
EUtest=EUtestdf.groupby('year_week',as_index=False).sum()

#(EUtest['year_week'].apply(str)).strip()

EUtest['date']=pd.to_datetime(EUtestdf.year_week.add('-0'), format='%Y-W%W-%w')-dt.timedelta(5)

from dataframe_utilities import ECDC_week2day

EUtestdaily=ECDC_week2day(EUtest,'tests_done','dailytests')
EUtestdaily['dailytestsavg']=EUtestdaily.dailytests.rolling(window=7,center=False).mean()

In [ ]:
EUtest

In [ ]:
EUtestdaily

In [ ]:
# Get total of Covid deaths
covid=df.groupby('date',as_index=False)[('deaths','positive')].agg('sum')
covid['date']=covid.date.dt.tz_localize(None)


fig, ax = plt.subplots(figsize=(15, 8))
plt.title('Official Covid numbers in Euromomo countries')
covid.plot(x='date',y='positive',ax=ax,color='blue',label='cases')#
covid.plot(x='date',y='deaths',ax=ax,color='red',label='deaths')#
plt.legend()
plt.ylabel('daily numbers')

fig.set_facecolor('w')

sns.despine;
plt.savefig('OfficialCovid_CasesDeaths.png',dpi=250)
plt.show()


# Get total of Covid deaths
#covid=df.groupby('date',as_index=False)[('deaths','positive')].agg('sum')
#covid['date']=covid.date.dt.tz_localize(None)


fig, ax = plt.subplots(figsize=(15, 8))
plt.title('Official Covid deaths in Euromomo countries')

covid.plot(x='date',y='deaths',ax=ax,color='red',legend=False)#
#plt.legend()
plt.ylabel('daily deaths')

fig.set_facecolor('w')

sns.despine;
plt.savefig('OfficialCovidDeaths.png',dpi=250)
plt.show()




# Get Euromomo data

In [ ]:
data=get_euromomo_data()
data.tail()

#  Plot Euromomo deaths & baseline graph

In [ ]:
def weekdiff(start, end):
    days = pd.to_datetime(end) - pd.to_datetime(start)
    week = int(pd.Timedelta(days).days / 7)
    remainder = pd.Timedelta(days).days % 7
    #print(week)
    return week



In [ ]:
yearlyintervals=[]
for i in np.arange(4):
    yearlyintervals.append(data.date.iloc[-1-i*52])
    
    
for yearlyinterval in yearlyintervals: print(yearlyinterval)
    
year_to_year_interval=[]
for i in np.arange(1,len(yearlyintervals)):

    year_to_year_interval.append(weekdiff(yearlyintervals[i],yearlyintervals[i-1]))
    
yearlyindices=data[data.date.isin(yearlyintervals)].index
    
#Find the cumulative sum of deaths for every year starting from latest date in series, going backwards


yearly_totaldeathsarray=[]


for i in np.arange(len(yearlyintervals)-1):
    yearly_totaldeaths=data.deaths.iloc[yearlyindices[i]:yearlyindices[i+1]].sum()
    yearly_totaldeathsarray.append(yearly_totaldeaths)
    
    

In [ ]:
yearly_totaldeathsarray

In [ ]:
data.to_pickle('Euromomodata_week50.pkl')

In [ ]:

fig, ax = plt.subplots(figsize=(15, 8))

data.plot(x="date", y="deaths",color='blue',ax=ax,legend=True,label='Deaths')


wrtbaseline=False
wrtpeaks=False
wrtdate=True


if wrtpeaks:
    
    plt.title('All-age deaths in EUROMOMO countries')

    for i in np.arange(len(troughs)):
        plt.axvline(data.date.iloc[troughs[i]],ymax=0.9,color='gray',alpha=0.5,linestyle=':')
    for i in np.arange(len(troughs)-1):
        ax.annotate('{} - {}'.format(intervaldates[i][0],intervaldates[i][1]), xy=(data.date.iloc[peaks[i]], data.deaths.iloc[troughs[i]]  ),
                    xytext=(data.date.iloc[troughs[i]+5], 1.05*data.deaths.iloc[peaks[i]]+6000),size=12)
        ax.annotate('{:,} weeks'.format(trough_to_trough_interval[i]), xy=(data.date.iloc[peaks[i]], data.deaths.iloc[troughs[i]]  ),
                    xytext=(data.date.iloc[troughs[i]+5], 1.05*data.deaths.iloc[peaks[i]]+3000))
        ax.annotate('{:,} deaths'.format(int(totaldeathsarray[i])), xy=(data.date.iloc[peaks[i]], data.deaths.iloc[troughs[i]]  ),
                    xytext=(data.date.iloc[troughs[i]+5], 1.05*data.deaths.iloc[peaks[i]]))
        
elif wrtbaseline:
    
    plt.title('All-age deaths in EUROMOMO countries \n with total deaths between troughs in baseline')
    
    for i in np.arange(len(baseline_troughs)):
        plt.axvline(data.date.iloc[baseline_troughs[i]],ymax=0.9,color='gray',alpha=0.5,linestyle=':')
    for i in np.arange(len(baseline_troughs)-1):
        ax.annotate('{} - {}'.format(baseline_intervaldates[i][0],baseline_intervaldates[i][1]),
                    xy=(data.date.iloc[baseline_peaks[i]], data.deaths.iloc[baseline_troughs[i]]  ),
                    xytext=(data.date.iloc[baseline_troughs[i]+5], 
                            1.1*data.deaths.iloc[baseline_peaks[i]]+6000),size=12)
        ax.annotate('{:,} weeks'.format(baseline_trough_to_trough_interval[i]),
                    xy=(data.date.iloc[baseline_peaks[i]], data.deaths.iloc[baseline_troughs[i]]  ),
                    xytext=(data.date.iloc[baseline_troughs[i]+5], 1.1*data.deaths.iloc[baseline_peaks[i]]+3000))
        ax.annotate('{:,} deaths'.format(int(baseline_totaldeathsarray[i])), 
                    xy=(data.date.iloc[baseline_peaks[i]], data.deaths.iloc[baseline_troughs[i]]  ),
                    xytext=(data.date.iloc[baseline_troughs[i]+5], 1.1*data.deaths.iloc[baseline_peaks[i]]))
        
elif wrtdate:
    
    plt.title('All-age deaths in EUROMOMO countries \n with total deaths for 52-week period')
    
    for i in np.arange(1,len(yearlyindices)):
        plt.axvline(data.date.iloc[yearlyindices[i-1]],ymax=0.9,color='gray',alpha=0.5,linestyle=':')
    for i in np.arange(1,len(yearlyindices)):
        ax.annotate('{} - {}'.format(data.date.iloc[yearlyindices[i-1]].strftime("%d %b %Y"),data.date.iloc[yearlyindices[i]].strftime("%d %b %Y")),
                    xy=(data.date.iloc[yearlyindices[i-1]], data.deaths.iloc[yearlyindices[i-1]] ),
                    xytext=(data.date.iloc[yearlyindices[i-1]+4], 
                            1.3*data.deaths.iloc[yearlyindices[i]]+6000),size=12)
#         ax.annotate('{:,} weeks'.format(baseline_trough_to_trough_interval[i]),
#                     xy=(data.date.iloc[baseline_peaks[i]], data.deaths.iloc[baseline_troughs[i]]  ),
#                     xytext=(data.date.iloc[baseline_troughs[i]+5], 1.1*data.deaths.iloc[baseline_peaks[i]]+3000))
        ax.annotate('{:,} deaths'.format(int(yearly_totaldeathsarray[i-1])), 
                    xy=(data.date.iloc[yearlyindices[i-1]], data.deaths.iloc[yearlyindices[i-1]] ),
                    xytext=(data.date.iloc[yearlyindices[i-1]+4], 1.28*data.deaths.iloc[yearlyindices[i]]))
            
    
#sns.set(font_scale=5) 
#sns.set_style("ticks", {"xtick.major.size": 8, "ytick.major.size": 8})




ax.xaxis.set_major_locator(mdates.YearLocator())

ax.xaxis.set_major_locator(mdates.MonthLocator(interval=3))

ax.xaxis.set_minor_locator(mdates.MonthLocator())


#plt.vlines(x=data.date.iloc[peaks], ymin=contour_heights, ymax=data.deaths.iloc[peaks])

#ax.xaxis.set_minor_locator(mdates.WeekdayLocator())
#set major ticks format
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))
ax.get_xaxis().set_tick_params(which='major', pad=5)

data.plot(x='date',y='baseline',color='orange',ax=ax,label='Euromomo baseline',alpha=0.5)

ax.fill_between(data.date,data.baseline_upper,data.baseline_lower, alpha=0.35,color='orange')


#x.plot(x="date", y="deaths",color='green',ax=ax,legend=False)


ax.legend(fancybox=True, framealpha=1, shadow=False, borderpad=1,
          loc='upper left',fontsize='small',facecolor='white')

fig.set_facecolor('w')

#plt.show()
plt.tight_layout();
sns.despine;


plt.savefig('plots/Euromomo_total_deaths.jpg',dpi=250)
 